# Lab 9 - Gradio and AI Assistant

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display
from agents import  WebSearchTool, function_tool, Agent, Runner
from agents.model_settings import ModelSettings
import gradio as gr
from IPython.display import display, Markdown
import requests
import json

ModuleNotFoundError: No module named 'agents'

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
serper_api_key = os.getenv('SERPER_API_KEY')
MODEL = "gpt-4.1-mini"
serper_url = "https://google.serper.dev/search"
openai = OpenAI()

In [ ]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999", "mumbai": "$2599"}
system_message = f"""You are a helpful assistant for an Airline called FlightAI. 
If relevent, Round trip fares to the certain cities are as follows: 
{ticket_prices}. If the city name is not listed, let the user know we dont serve that city yet ad show them the list of cities we do serve. 
If asked for one way prices to these cities, cut the listed prices in half.
Give short, witty, snarky answers, no more than 1 sentence. """

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [ ]:
import base64
from io import BytesIO
from PIL import Image

def artist_agent(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [ ]:
def chat(history):
    message = history[-1]["content"]
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    lower_message = message.lower()
    city = next((name for name in ticket_prices if name in lower_message), None)
    image = artist_agent(city.title()) if city else None
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    return history, image


In [ ]:
# More involved Gradio code as we're not using the preset Chat interface!

with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=400, type="messages")
        image_output = gr.Image(height=400)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")

    def do_entry(message, history):
        history += [{"role": "user", "content": message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat, inputs=chatbot, outputs=[chatbot, image_output]
    )

ui.launch()

In [ ]:
def search_web(web_query):
    print(f"search web: {web_query}")
    payload = json.dumps({"q": web_query })
    headers = {
    "X-API-KEY": serper_api_key,
    "Content-Type": "application/json"
    }
     
    response = requests.request("POST", serper_url, headers=headers, data=payload)
    print (response.text)
    return response.text

def search_the_web(to_city, from_city="San Francisco", extra_info=""):
    
    return search_web( "Find the cheapest round trip filght prices from " + from_city + " to " + to_city + " in the next week.")
        


search_the_web_json = {
    "name": "search_the_web",
    "description": "Use this tool to search flight prices",
    "parameters": {
        "type": "object",
        "properties": {
            "to_city": {
                "type": "string",
                "description": "Destination City"
            },
            "from_city": {
                "type": "string",
                "description": "Originating city"
            },
            "extra_info": {
                "type": "string",
                "description": "Any additional information about the search that's worth recording to give context"
            }
        },
        "required": ["to_city"],
        "required": ["extra_info"],
        "additionalProperties": False
    }
}
    
tools = [{"type": "function", "function": search_the_web_json}]

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:

new_system_message = f"""You are a helpful assistant for an Airline called FlightAI. 
If asked about the ticket prices to any city and if you dont know the latest prices, use e search_the_web tool to search the web and use the response to 
find the cheapest flight from San Francisco. Assume all the flights originate from San Francisco
If asked for one way prices to these cities, cut the listed prices in half. If price is not found, let the user  know that you couldnt fine a price to give city, 
Give short, witty, snarky answers, no more than 1 sentence. """

def chat_with_web(history):
    message = history[-1]["content"]
   
    messages = [{"role": "system", "content": new_system_message}] + history
    lower_message = message.lower()
    
    
    done = False
    while not done:
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
        print (response.choices[0])
        finish_reason = response.choices[0].finish_reason
        print (f"finish reason"  + finish_reason)
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
        
    city = next((name for name in ticket_prices if name in lower_message), None)
    
    image = artist_agent(city.title()) if city else None
    
    
    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    return history, image

In [ ]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=400, type="messages")
        image_output = gr.Image(height=400)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")

    def do_entry(message, history):
        history += [{"role": "user", "content": message}]
        return "", history

    entry.submit(do_entry, inputs=[entry, chatbot], outputs=[entry, chatbot]).then(
        chat_with_web, inputs=chatbot, outputs=[chatbot, image_output]
    )

ui.launch()

## Congratulations!

You just created a multi-modal AI Assistant

### Simple Assignment

Based on this code:

```python
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "sydney": "$2999"}
```

Allow the Chatbot to quote ticket prices for any of those destinations, and show images!

## MAJOR ASSIGNMENT

Make a variation of this Chatbot that applies this to your business


### Optional Stretch Assignment

Research "tool use" (also known as Function Calling) and then use this technique to add Tool capabilities to your LLM to look up ticket prices..


## Additional preparation for next week:

Please gather information about yourself for our RAG example, such as resume, personal projects, any relevant artifacts.